In [8]:
import numpy as np 
import h5py
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
%matplotlib ipympl

In [3]:
dNgrp = np.loadtxt('/home/qyli/work/castic_MAR/data/groups/DESIDR9_NGC_group')
dSgrp = np.loadtxt('/home/qyli/work/castic_MAR/data/groups/DESIDR9_SGC_group')

In [7]:
idxN = np.where(dNgrp[:,-2] > 14)[0]
idxS = np.where(dSgrp[:,-2] > 14)[0]
print(idxN.shape, idxS.shape)
idxN.shape[0] + idxS.shape[0]

(85264,) (77026,)


162290

In [9]:
'''
column 1: group ID
column 2: richness
column 3: ra
column 4: dec
column 5: redshift
column 6: log M_h/(M_sun/h)
column 7: L_group (10^10Lsun/h/h)
'''
dgroup = np.loadtxt('../odata/CLAUDS_HSC_iband_group')
d2 = np.loadtxt('../odata/iCLAUDS_HSC_iband_2')
digal = np.loadtxt('../odata/CLAUDS_HSC_iband_igal')

In [10]:
'''
plot group images
#------------------
idx_group: idx of group in xxx_group file
path_image: the path to input image file
out_image: the path to output image file
'''

def find_image(zr,nm,nchoose):
    from astropy.wcs import WCS
    import astropy.units as u
    from astropy.coordinates import SkyCoord
    import glob
    import numpy as np
    
    path_image = '../data/Pc_z%s_Ng%s/z%s-Ng%s' %(zr,nm,zr,nm)
    filename0 = glob.glob(path_image+'/%s-cutout-HSC-*' %int(nchoose+2))

    if len(filename0) < 1:
        print('No this group image')
    image_file = get_pkg_data_filename(filename0[0])
    print(fits.getdata(image_file, ext=0).shape)

    hdu = fits.open(image_file)
    wcs = WCS(hdu[1].header)
    return wcs

def plot_one_group(nclu,nchoose,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag,rci):
    from astropy.wcs import WCS
    import astropy.units as u
    from astropy.coordinates import SkyCoord
    import glob
    import numpy as np
    from mpl_toolkits.axes_grid1 import make_axes_locatable

    #choose image pathe
    
    #prepare group and member galaxy information
    print("The group id is ", nclu)
    clu_ra = dgroup[nclu-1,2]
    clu_dec = dgroup[nclu-1,3]
    clu_redz = dgroup[nclu-1,4]
    clu_Mh = dgroup[nclu-1,-2].round(2)
    
    idx_gal = np.where(d2[:,0] == nclu)[0]
    galid = np.int64(d2[idx_gal,1])
    ra_gal = digal[galid - 1, 1]
    dec_gal = digal[galid - 1, 2]
    redz_gal = digal[galid - 1, 3].round(2)

    #galaxis around group center
    dra = 0.09
    ddec = 0.09
    idx_region = np.where((clu_ra > digal[:,1] - dra) & (clu_ra < digal[:,1] + dra) & 
                          (clu_dec > digal[:,2] - ddec) & (clu_dec < digal[:,2] + ddec)
                         & (np.abs(clu_redz - digal[:,3]) < 0.1))[0] 

    idx_around = list(idx_region)
    gglistid = list(galid-1)
    for nn in range(galid.shape[0]):
        if gglistid[nn] in idx_around:
            idx_around.remove(gglistid[nn])
    ra_reg = digal[idx_around,1]
    dec_reg = digal[idx_around,2]
    redz_reg = digal[idx_around,3]
    
    #plot around galaxies
#     sc2 = ax.scatter(ra_reg, dec_reg,transform=ax.get_transform('world'), s = 30, c = redz_reg,
#              cmap = 'jet', zorder = 5, vmin = 0, vmax = 6, alpha = 0.6)
    sc2 = ax.scatter(ra_reg, dec_reg,transform=ax.get_transform('world'), s = 8, c = 'r', edgecolor = 'none',
              zorder = 5, alpha = 0.45, label = r'$N_{\rm nei}$ = %s' %(Nag))
#     sc2.set_facecolor('none')

    print(ra_reg.shape)

    #prepare group r180 
    import linecache
    fprop = '../odata/CLAUDS_HSC_iband_prop'
    nline = 1+(nclu-1)*6
    gprop = linecache.getline(fprop,nline)
    phyp = [float(prop) for prop in gprop.split()]
    r180 = phyp[-2] #unit:Mpc/h
    
    #=====================================================================
    #begin plot
    # image_file = get_pkg_data_filename('../../images_HSC_CLAUDS/Deep/calexp-HSC-I-17270-1,8.fits')
    # image_file = get_pkg_data_filename('../data/images/cutout-HSC-I-17406-pdr2_dud-210625-033129.fits')
    path_image = '../data/Pc_z%s_Ng%s/z%s-Ng%s' %(zr,nm,zr,nm)
    filename0 = glob.glob(path_image+'/%s-*' %int(nchoose+2))
    if len(filename0) <1:
        print('No this group image')
    image_file = get_pkg_data_filename(filename0[0])
    print(fits.getdata(image_file, ext=0).shape)

    hdu = fits.open(image_file)
    ax.imshow(hdu[1].data, cmap = 'Greys', vmax = mapmax, vmin = mapmin, origin='lower')

    #plot center
    cen = wcs.wcs_world2pix(clu_ra, clu_dec, 0)
    ax.plot(cen[0],cen[1],'*',c  = 'r',zorder = 6,label = 'center: $z$ = %s' %clu_redz.round(2))


    #plot member galaxy
    ax.plot(ra_gal, dec_gal, 's', transform=ax.get_transform('world'),
            mfc='none',mec= 'r', ms = 6, lw = 0.8, zorder = 5, 
            label = 'members: $N_\mathrm{g}$ = %s' %ra_gal.shape[0])
    
#     ax.scatter(ra_gal, dec_gal,transform=ax.get_transform('world'), s = 30, c = 'r',
#               zorder = 5, alpha = 0.4)

    #plot group region with r180
    #convert r180 to deg
    from astropy.cosmology import FlatLambdaCDM
    import numpy as np
    from matplotlib.patches import Circle
    
    cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)
    d_A = cosmo.comoving_distance(z=clu_redz)
    r180u =  r180 * u.Mpc / 0.674
    theta_radian = (r180u / d_A).to(u.degree, u.dimensionless_angles()).value # unit is Mpc only now
    print(theta_radian)
    
    regr180 = Circle((clu_ra, clu_dec), theta_radian, ec='magenta', ls = '--', lw = 1.5, fc='none', zorder = 5,
               transform=ax.get_transform('world'))
    ax.add_patch(regr180)
    
    theta_radian2mpc = (2*u.Mpc / 0.674 / d_A).to(u.degree, u.dimensionless_angles()).value # unit is Mpc only now
    regr2mpc = Circle((clu_ra, clu_dec), theta_radian2mpc, ec='blue', ls = ':', lw = 1.5, fc='none', zorder = 5,
               transform=ax.get_transform('world'))
    ax.add_patch(regr2mpc)
    
    
    theta_radian4mpc = (4*u.Mpc / 0.674 / d_A).to(u.degree, u.dimensionless_angles()).value # unit is Mpc only now
    regr4mpc = Circle((clu_ra, clu_dec), theta_radian4mpc, ec='blue', ls = '--', lw = 1.5, fc='none', zorder = 5,
               transform=ax.get_transform('world'))
    ax.add_patch(regr4mpc)
    
#     theta_radian6mpc = (6*u.Mpc / 0.674 / d_A).to(u.degree, u.dimensionless_angles()).value # unit is Mpc only now
#     regr6mpc = Circle((clu_ra, clu_dec), theta_radian6mpc, ec='blue', ls = '-', lw = 1.5, fc='none', zorder = 5,
#                transform=ax.get_transform('world'))
#     ax.add_patch(regr6mpc)
    
    theta_radiancimpc = (rci*u.Mpc / 0.674 / d_A).to(u.degree, u.dimensionless_angles()).value # unit is Mpc only now
    regrcimpc = Circle((clu_ra, clu_dec), theta_radiancimpc, ec='r', ls = '-', lw = 1.5, fc='none', zorder = 5,
               transform=ax.get_transform('world'))
    ax.add_patch(regrcimpc)
    
   #plot around group center
    ddra = np.abs(dgroup[:,2] - clu_ra)
    dddec = np.abs(dgroup[:,3] - clu_dec)
    ddredz = np.abs(dgroup[:,4] - clu_redz)
    
    idx_region = np.where((ddra < 0.09) & (dddec < 0.09) & (ddredz < 0.1) & (dgroup[:,1] > 1) & (ddra != 0 ))[0] 

    ra_regc = dgroup[idx_region, 2]
    dec_regc = dgroup[idx_region, 3]
    redz_regc = dgroup[idx_region, 4]
    print(len(idx_region))
    if len(idx_region):
        sc2 = ax.scatter(ra_regc, dec_regc, transform=ax.get_transform('world'), s = 10, marker = '*', c = 'blue',
                         zorder = 5, facecolor = 'none', alpha = 0.8)

    #set xlabel
    ax.coords.grid('icrs',color = 'none')
    ax.coords[0].set_axislabel('RA')
    ax.coords[1].set_axislabel('Dec')
    ax.coords[0].set_ticklabel(size="small")
    ax.coords[1].set_ticklabel(size="small")

    lon = ax.coords[0]
    lon.set_major_formatter('d.dd')
    lat = ax.coords[1]
    lat.set_major_formatter('d.dd')
    lat.set_ticks_position('l')
    lon.set_ticks_position('b')
    
    ax.set_xlim(cen[0]-ngrid,cen[0]+ngrid)
    ax.set_ylim(cen[1]-ngrid,cen[1]+ngrid)
    ax.legend(fontsize = 'x-small', loc = 1,shadow = True)

## z >= 2, Ng >= 10

In [26]:
fig = plt.figure(figsize = (12, 12))

gid = [47826, 61254, 68057] #z >=2, Ng >= 10

zr = 2; nm = 10;ngrid = 1800;rci = 5;
Nag = 403; cag = 0.23;
wcs = find_image(zr,nm,0)
ax = fig.add_subplot(331,projection=wcs)
mapmax = 0.26; mapmin = -0.1;
plot_one_group(gid[0],0,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag,rci)
plt.text(150.966, 1.497, 'High', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

Nag = 121; cag = 0.32;
wcs = find_image(zr,nm,1)
ax = fig.add_subplot(332,projection=wcs)
mapmax = 0.25; mapmin = -0.1;
plot_one_group(gid[1],1,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag,rci)
plt.text(150.11, 2.01, 'Medium', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

Nag = 60; cag = 0.33;
wcs = find_image(zr,nm,2)
ax = fig.add_subplot(333,projection=wcs)
mapmax = 0.25; mapmin = -0.1;
plot_one_group(gid[2],2,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag,rci)
plt.text(149.72, 3.748, 'Low', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

#=========================================
gid = [192242, 104820, 186417] # z >= 3, Ng >= 5

zr = 3; nm = 5;ngrid = 1800;rci = 6;
Nag = 248; cag = 0.22; 
wcs = find_image(zr,nm,0)
ax = fig.add_subplot(334,projection=wcs)
mapmax = 0.26; mapmin = -0.1;
plot_one_group(gid[0],0,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag,rci)
plt.text(151.463, 2.904, 'High', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

Nag = 78; cag = 0.31;
wcs = find_image(zr,nm,1)
ax = fig.add_subplot(335,projection=wcs)
mapmax = 0.25; mapmin = -0.1;
plot_one_group(gid[1],1,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag,rci)
plt.text(36.126, -4.802, 'Medium', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

Nag = 18; cag = 0.33;
wcs = find_image(zr,nm,2)
ax = fig.add_subplot(336,projection=wcs)
mapmax = 0.25; mapmin = -0.1;
plot_one_group(gid[2],2,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag,rci)
plt.text(149.60, 2.694, 'Low', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

#==========================================
gid = [321925, 354967, 297062] # z >= 4, Ng >= 3

zr = 4; nm = 3;ngrid = 1800;rci = 7;
Nag = 61; cag = 0.32;
wcs = find_image(zr,nm,0)
ax = fig.add_subplot(337,projection=wcs)
mapmax = 0.26; mapmin = -0.1;
plot_one_group(gid[0],0,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag,rci)
plt.text(149.77, 1.855, 'High', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

Nag = 23; cag = 0.19; 
wcs = find_image(zr,nm,1)
ax = fig.add_subplot(338,projection=wcs)
mapmax = 0.25; mapmin = -0.1;
plot_one_group(gid[1],1,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag,rci)
plt.text(352.857, 0.642, 'Medium', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

Nag = 3; cag = 0.22;
wcs = find_image(zr,nm,2)
ax = fig.add_subplot(339,projection=wcs)
mapmax = 0.25; mapmin = -0.1;
plot_one_group(gid[2],2,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag,rci)
plt.text(36.581, -5.16, 'Low', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

plt.show()
plt.subplots_adjust(top=0.98,left = 0.09, right = 0.96, bottom = 0.05,hspace = 0.02,wspace = 0.24)

plt.savefig('../figs/eg_Pc.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(4287, 4287)
The group id is  47826
(618,)
(4287, 4287)
0.008677850967278385
77
(4286, 4287)
The group id is  61254
(214,)
(4286, 4287)
0.00831542245824386
29
(4284, 4290)
The group id is  68057
(106,)
(4284, 4290)
0.010404149548138742
14
(4286, 4289)
The group id is  192242
(368,)
(4286, 4289)
0.005498024417112949
48
(4284, 4289)
The group id is  104820
(109,)
(4284, 4289)
0.006163276132397826
11
(4284, 4289)
The group id is  186417
(34,)
(4284, 4289)
0.004586274920443429
3
(4286, 4289)
The group id is  321925
(73,)
(4286, 4289)
0.00398127241611897
6
(4287, 4287)
The group id is  354967
(27,)
(4287, 4287)
0.004049412993581301
0
(4292, 4283)
The group id is  297062
(4,)
(4292, 4283)
0.003408924777593267
0


## z >= 3, Nag >= 4

In [31]:
fig = plt.figure(figsize = (12, 4.5))

gid = [174401, 105412, 104820]

zr = 3; nm = 5;ngrid = 1800;
Nag = 223; cag = 0.22;
wcs = find_image(zr,nm,0)
ax = fig.add_subplot(131,projection=wcs)
mapmax = 0.26; mapmin = -0.1;
plot_one_group(gid[0],0,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag)
plt.text(151.45, 3.03, 'High', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

Nag = 175; cag = 0.31;
wcs = find_image(zr,nm,1)
ax = fig.add_subplot(132,projection=wcs)
mapmax = 0.25; mapmin = -0.1;
plot_one_group(gid[1],1,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag)
plt.text(352.465, -1.265, 'Medium', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

Nag = 78; cag = 0.33;
wcs = find_image(zr,nm,2)
ax = fig.add_subplot(133,projection=wcs)
mapmax = 0.25; mapmin = -0.1;
plot_one_group(gid[2],2,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag)
plt.text(36.13, -4.803, 'Low', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

plt.show()
plt.subplots_adjust(top=0.98,left = 0.09, right = 0.96, bottom = 0.05,hspace = 0.04,wspace = 0.24)

# plt.savefig('../figs/z3_Ng5.pdf')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(4287, 4288)
The group id is  174401
(343,)
(4287, 4288)
0.006668631177636394
56
(4287, 4287)
The group id is  105412
(267,)
(4287, 4287)
0.006169691029574232
47
(4284, 4289)
The group id is  104820
(109,)
(4284, 4289)
0.006163276132397826
11


## z >= 4, Nag >= 3

In [32]:
fig = plt.figure(figsize = (12, 4.5))

gid = [305505, 354967, 362628]

zr = 4; nm = 3;ngrid = 1600;
Nag = 32; cag = 0.32;
wcs = find_image(zr,nm,0)
ax = fig.add_subplot(131,projection=wcs)
mapmax = 0.26; mapmin = -0.1;
plot_one_group(gid[0],0,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag)
plt.text(150.34, 1.80, 'High', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

Nag = 16; cag = 0.19;
wcs = find_image(zr,nm,1)
ax = fig.add_subplot(132,projection=wcs)
mapmax = 0.25; mapmin = -0.1;
plot_one_group(gid[1],1,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag)
plt.text(352.85, 0.645, 'Medium', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

Nag = 1; cag = 0.22;
wcs = find_image(zr,nm,2)
ax = fig.add_subplot(133,projection=wcs)
mapmax = 0.25; mapmin = -0.1;
plot_one_group(gid[2],2,zr,nm,ax,mapmin,mapmax,ngrid,Nag,cag)
plt.text(37.09, -3.92, 'Low', fontsize = 15,
                 transform=ax.get_transform('world'),zorder = 5)

plt.show()
plt.subplots_adjust(top=0.98,left = 0.09, right = 0.96, bottom = 0.05,hspace = 0.04,wspace = 0.24)

# plt.savefig('../figs/z4_Ng3.pdf')




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(4288, 4287)
The group id is  305505
(64,)
(4288, 4287)
0.004530824500758204
2
(4287, 4287)
The group id is  354967
(27,)
(4287, 4287)
0.004049412993581301
0
(4288, 4286)
The group id is  362628
(3,)
(4288, 4286)
0.0029288497696828147
0
